<a href="https://colab.research.google.com/github/FreddyMS/COP4777-Data-Mining-Course-Fall-2022/blob/main/FinalCopyofAmazonReviewProjectCNN.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
#Work worked with Yelise on buiding the program from scratch and debugging
#Unfornuately we ran out of time to fit the file format which prob was the easiest
#part of the project D:
import numpy as np
import pandas as pd
#Work this
#opening my files, setting train, test variables
!wget http://34.82.108.37/amazonReviewsTrain.csv.zip
!wget https://github.com/zacharski/ml-class/raw/master/data/fasttext.zip
!ls
!unzip fasttext.zip 


Amazon_train_reviews=pd.read_csv("amazonReviewsTrain.csv.zip")
Amazon_test_reviews= pd.read_csv("http://34.82.108.37/amazonReviewsTest.csv")
Amazon_test_reviews #11 columns NOT include rating  (positive, neutral,negative)
Amazon_train_reviews# 12 columns including rating (positive, neutral,negative)
Amazon_train_reviews=Amazon_train_reviews.drop(["style", "image","reviewerID","unixReviewTime","asin", "vote", "verified",	"reviewTime", "reviewerName"	], axis=1)
Amazon_test_reviews=Amazon_test_reviews.drop(["style","reviewerName", "image","reviewerID","unixReviewTime","asin", "vote", "verified",	"reviewTime"], axis=1)

--2022-12-10 03:09:33--  http://34.82.108.37/amazonReviewsTrain.csv.zip
Connecting to 34.82.108.37:80... connected.
HTTP request sent, awaiting response... 200 OK
Length: 39456705 (38M) [application/zip]
Saving to: ‘amazonReviewsTrain.csv.zip’

amazonReviewsTrain. 100%[===================>]  37.63M  16.1MB/s    in 2.3s    

2022-12-10 03:09:35 (16.1 MB/s) - ‘amazonReviewsTrain.csv.zip’ saved [39456705/39456705]

--2022-12-10 03:09:35--  https://github.com/zacharski/ml-class/raw/master/data/fasttext.zip
Resolving github.com (github.com)... 20.27.177.113
Connecting to github.com (github.com)|20.27.177.113|:443... connected.
HTTP request sent, awaiting response... 302 Found
Location: https://raw.githubusercontent.com/zacharski/ml-class/master/data/fasttext.zip [following]
--2022-12-10 03:09:36--  https://raw.githubusercontent.com/zacharski/ml-class/master/data/fasttext.zip
Resolving raw.githubusercontent.com (raw.githubusercontent.com)... 185.199.108.133, 185.199.109.133, 185.199.110.133,

/usr/local/lib/python3.8/dist-packages/IPython/core/interactiveshell.py:3326: DtypeWarning: Columns (8) have mixed types.Specify dtype option on import or set low_memory=False.
  exec(code_obj, self.user_global_ns, self.user_ns)


In [ ]:
#Amazon_train_reviews.drop(["style","reviewerName"],axis=1)
Amazon_train_reviews#221391 rows
Amazon_train_reviews.reviewText[1] #10k rows
Amazon_test_reviews

,reviewText,summary
0,I am a (very amateur) EDM and Hip Hop Producer...,Extremely Happy.
1,These are pretty beefy strings even for black ...,"once again, Rotosound strings fail to impress ..."
2,"I've played every new ""improved"" string out th...",and keep coming home to the girls that really ...
3,"I thought I would give this pedal a try, becau...",save up for a better pedal
4,Been wanting to try one for over a year. Good...,Big Six
...,...,...
9995,I took it off my guitar! I would rather look a...,Save your money!!
9996,Better than the stock part on non-USA made gui...,Four Stars
9997,I'm updating this review after using the violi...,First Violin Purchase
9998,A lot of sound research and engineering go int...,Another winner from Hardwire!


In [ ]:
from sklearn.preprocessing import OneHotEncoder
ohe = OneHotEncoder(handle_unknown='ignore')

data_label =Amazon_train_reviews.rating
data_label= pd.get_dummies(data_label)
Amazon_train_reviews = Amazon_train_reviews.drop("rating", axis=1)
#Amazon_train_reviews.rating.unique()

data_text =  Amazon_train_reviews['summary'] + ' ' + Amazon_train_reviews['reviewText'] 
data_label
data_text= data_text.map(str)



In [ ]:
from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 20000)
#build the word index
tokenizer.fit_on_texts(data_text)
# now convert the words to integers
data_sequences = tokenizer.texts_to_sequences(data_text)
data_sequences[0]

[45, 33, 19, 547, 9, 25, 13, 2206, 18, 560]

In [ ]:
tokenizer.sequences_to_texts([data_sequences[1000]])

['five stars good']

In [ ]:



# Cut texts after this number of words 
# (among top max_features most common words)
maxlen = 50


from keras.utils import pad_sequences


# This turns our lists of integers
# into a 2D integer tensor of shape `(samples, maxlen)`
data50 = pad_sequences(data_sequences, maxlen=maxlen)
data50[0]


array([   0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,    0,    0,    0,    0,
          0,    0,    0,    0,    0,    0,    0,   45,   33,   19,  547,
          9,   25,   13, 2206,   18,  560], dtype=int32)

In [ ]:
#100 percent of the data was used
from sklearn.model_selection import train_test_split
train50, test50, train_labels, test_labels = train_test_split(data50, data_label)
test_labels

,negative,neutral,positive
84644,0,0,1
195644,1,0,0
148298,0,0,1
206395,0,0,1
210560,0,0,1
...,...,...,...
213290,0,0,1
126181,0,0,1
80171,0,0,1
29204,0,0,1


In [ ]:
#open the glove file and specifing which file
import numpy as np
import pandas as pd
!curl http://zacharski.org/files/courses/cs419/glove.6B.100d.zip > glove.6B.100d.zip
!ls

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100  128M  100  128M    0     0  13.0M      0  0:00:09  0:00:09 --:--:-- 16.1M
amazonReviewsTrain.csv.zip  fasttext.zip       sample_data
fasttext.vec		    glove.6B.100d.zip


In [ ]:
!unzip glove.6B.100d.zip

Archive:  glove.6B.100d.zip
  inflating: glove.6B.100d.txt       


In [ ]:
from keras.layers import Embedding

# The Embedding layer takes at least two arguments:
# the number of possible tokens, here 1000 (1 + maximum word index),
# and the dimensionality of the embeddings, here 64.
embedding_layer = Embedding(1000, 64)

In [ ]:
#  Open glove file, setting, coefficients, matrix index, finding the length of the embedded index
embeddings_index = {}
f = open('glove.6B.100d.txt')
for line in f:
    values = line.split()
    word = values[0]
    coefs = np.asarray(values[1:], dtype='float32')
    embeddings_index[word] = coefs
f.close()

print('Found %s word vectors.' % len(embeddings_index))

embedding_dim = 100
max_words = 20000
word_index = tokenizer.word_index

embedding_matrix = np.zeros((max_words, embedding_dim))
for word, i in word_index.items():
    embedding_vector = embeddings_index.get(word)
    if i < max_words:
        if embedding_vector is not None:
            # Words not found in embedding index will be all-zeros.
            embedding_matrix[i] = embedding_vector



Found 400000 word vectors.


In [ ]:
from keras.models import Sequential
from keras.layers import Embedding, Flatten, Dense
from keras.models import Sequential
from keras.layers import Flatten, Dense, Embedding
from keras import layers
#Setting a more complex network inhopes to get better results
#Setting input, output,flattening, densing, layering it with embedded matrix
cnn_model = Sequential()
cnn_model.add(Embedding(max_words, embedding_dim, input_length=maxlen))
cnn_model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
cnn_model.add(layers.Conv1D(filters=64, kernel_size=3, activation='relu'))
cnn_model.add(layers.Dropout(0.5))
cnn_model.add(layers.MaxPooling1D(pool_size=2))
cnn_model.add(Flatten())
cnn_model.add(Dense(32, activation='relu'))
cnn_model.add(Dense(3, activation='softmax'))
cnn_model.summary()
cnn_model.layers[0].set_weights([embedding_matrix])
cnn_model.layers[0].trainable = True


Model: "sequential"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 embedding_1 (Embedding)     (None, 50, 100)           2000000   
                                                                 
 conv1d (Conv1D)             (None, 48, 64)            19264     
                                                                 
 conv1d_1 (Conv1D)           (None, 46, 64)            12352     
                                                                 
 dropout (Dropout)           (None, 46, 64)            0         
                                                                 
 max_pooling1d (MaxPooling1D  (None, 23, 64)           0         
 )                                                               
                                                                 
 flatten (Flatten)           (None, 1472)              0         
                                                        

In [ ]:
#Compiling my cnn model, setting loss to categorical crossentrophy since its non_binary
cnn_model.compile(optimizer='rmsprop',
              loss='categorical_crossentropy',
              metrics=['acc'])
history = cnn_model.fit(train50, train_labels,
                    epochs=10,
                    batch_size=32,
                     validation_split=0.2,
      validation_steps=50)


Epoch 1/10
4152/4152 [==============================] - 26s 4ms/step - loss: 0.3568 - acc: 0.8787 - val_loss: 0.3290 - val_acc: 0.8881
Epoch 2/10
4152/4152 [==============================] - 17s 4ms/step - loss: 0.3126 - acc: 0.8940 - val_loss: 0.3259 - val_acc: 0.8944
Epoch 3/10
4152/4152 [==============================] - 17s 4ms/step - loss: 0.2971 - acc: 0.9010 - val_loss: 0.3088 - val_acc: 0.9075
Epoch 4/10
4152/4152 [==============================] - 17s 4ms/step - loss: 0.2949 - acc: 0.9029 - val_loss: 0.2743 - val_acc: 0.9050
Epoch 5/10
4152/4152 [==============================] - 17s 4ms/step - loss: 0.2969 - acc: 0.9038 - val_loss: 0.2945 - val_acc: 0.9081
Epoch 6/10
4152/4152 [==============================] - 17s 4ms/step - loss: 0.2957 - acc: 0.9051 - val_loss: 0.3487 - val_acc: 0.9025
Epoch 7/10
4152/4152 [==============================] - 17s 4ms/step - loss: 0.2935 - acc: 0.9064 - val_loss: 0.3207 - val_acc: 0.9013
Epoch 8/10
4152/4152 [==============================] -

1730/1730 [==============================] - 4s 2ms/step - loss: 0.2897 - acc: 0.9108
Accuracy 0.910783    Loss: 0.289665


'\ntest_loss, test_acc, = model.evaluate(test50, test_labels)\nprint("Accuracy %f    Loss: %f" % (test_acc, test_loss))'

In [ ]:
#Structuring the Amazon dataset by placing summary and reviewtext into a single string
Amazon_test_reviews= pd.read_csv("http://34.82.108.37/amazonReviewsTest.csv")
Amazon_test_reviews=Amazon_test_reviews.drop(["style","reviewerName", "image","reviewerID","unixReviewTime","asin", "vote", "verified","reviewTime"], axis=1)
data_text =  Amazon_test_reviews['summary'] + ' ' + Amazon_test_reviews['reviewText'] 
data_text= data_text.map(str)
data_text

0       Extremely Happy. I am a (very amateur) EDM and...
1       once again, Rotosound strings fail to impress ...
2       and keep coming home to the girls that really ...
3       save up for a better pedal I thought I would g...
4       Big Six Been wanting to try one for over a yea...
                              ...                        
9995    Save your money!! I took it off my guitar! I w...
9996    Four Stars Better than the stock part on non-U...
9997    First Violin Purchase I'm updating this review...
9998    Another winner from Hardwire! A lot of sound r...
9999    Five Stars This is my second stand like this. ...
Length: 10000, dtype: object

In [ ]:


from keras.preprocessing.text import Tokenizer
tokenizer = Tokenizer(num_words = 20000)
#build the word index
tokenizer.fit_on_texts(data_text)
# now convert the words to integers
data_sequences = tokenizer.texts_to_sequences(data_text)
data_sequences[0]

maxlen = 50


from keras.utils import pad_sequences


# This turns our lists of integers
# into a 2D integer tensor of shape `(samples, maxlen)`
data50TestSet = pad_sequences(data_sequences, maxlen=maxlen)
data50TestSet[3]
#amazonReviewsPredictions = model.predict(data50TestSet)

array([   0,    0,    0,    0,    0,  663,   47,    7,    2,   77,   73,
          3,  320,    3,   59,  264,   10,   73,    2,  255,  105,    6,
         29,  558,   16,   22, 1887,   15,   68,   51,   60,   15,  550,
          7,  255,    5,  262,   50,    4,   51,    2,   77,   57,   73,
         15,   55,   34, 1231,   80,  185], dtype=int32)

In [ ]:
#predicting the probability using my fined tuned model ona new dataset(test set)
amazonreviewstest =cnn_model.predict(data50TestSet)
#try
print(amazonreviewstest)


313/313 [==============================] - 0s 2ms/step
[[8.5013984e-03 1.5240869e-02 9.7625774e-01]
 [1.7517211e-02 4.0316720e-02 9.4216609e-01]
 [6.9136379e-35 0.0000000e+00 1.0000000e+00]
 ...
 [2.0510554e-02 4.9945176e-02 9.2954427e-01]
 [1.4025621e-02 2.9856626e-02 9.5611775e-01]
 [1.0734198e-06 5.2398985e-07 9.9999845e-01]]


In [ ]:


import pandas as pd
import numpy as np

prediction = pd.DataFrame(amazonreviewstest, columns=['negative', 'neutral', 'positive']).to_csv('prediction.csv')







TypeError: ignored

In [ ]:
Amazon_test_reviews

,reviewText,summary
0,I am a (very amateur) EDM and Hip Hop Producer...,Extremely Happy.
1,These are pretty beefy strings even for black ...,"once again, Rotosound strings fail to impress ..."
2,"I've played every new ""improved"" string out th...",and keep coming home to the girls that really ...
3,"I thought I would give this pedal a try, becau...",save up for a better pedal
4,Been wanting to try one for over a year. Good...,Big Six
...,...,...
9995,I took it off my guitar! I would rather look a...,Save your money!!
9996,Better than the stock part on non-USA made gui...,Four Stars
9997,I'm updating this review after using the violi...,First Violin Purchase
9998,A lot of sound research and engineering go int...,Another winner from Hardwire!


In [ ]:
file1 = open("amazonReviewsPredictions.txt", "w")
 
# Reading from file
def test():
  i = 0
  answers = ['negative', 'neutral', 'positive']
  with open('amazonReviewsPredictions.txt', 'r') as outfile:
    lines = outfile.readlines()
    for line in lines:
        if line.strip() not in answers:
          return("invalid entry line %i" % (i))
        i += 1
    if i != 10000:
      return ("incorrect number of lines")
    else:
      return ("file looks ok")
print(test())

 
file1.close()



incorrect number of lines
